In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# **Data Preprocessing**

In [ ]:
df_orig=pd.read_csv('/kaggle/input/insurance/insurance.csv')
df=df_orig.copy()
df_orig.info()

In [ ]:
df['region'].value_counts()

In [ ]:
df['smoker'].value_counts()

In [ ]:
df['sex'].value_counts()

# Categorical Variable****

In [ ]:
df['region']=df['region'].replace(to_replace =["southeast","southwest","northwest", "northeast"], value = [1,2,3,4])

In [ ]:
df['region'].value_counts()

In [ ]:
df['smoker']=pd.get_dummies(df['smoker'],drop_first=True)

In [ ]:
df['sex']=pd.get_dummies(df['sex'],drop_first=True)

In [ ]:
df["charges"] = df['charges'].astype('int')
df["bmi"] = df['charges'].astype('int')

In [ ]:
df.info()

In [ ]:
df_clean=df.copy()

# **# Outlier Hadling******

In [ ]:
for i in df_clean:
    print(i)
    print(np.where(np.abs(stats.zscore(df_clean[i]))>3))

In [ ]:
outlier=['bmi','children','charges']

lower=[0]*len(outlier)
upper=[0]*len(outlier)

#Identifying Outlier
for index, x in enumerate (outlier) :
    lower[index]=df_clean[x].quantile(0.1)
    upper[index]=df_clean[x].quantile(0.9)
print(lower)
print(upper)


In [ ]:
for index, i in enumerate (outlier):
    df_clean[i] = np.where(df_clean[i] <lower[index],lower[index],df_clean[i])
    df_clean[i] = np.where(df_clean[i] >upper[index],upper[index],df_clean[i])
 

In [ ]:
for i in df_clean:
    print(i)
    print(np.where(np.abs(stats.zscore(df_clean[i]))>3))

# **DEFINING INDEPENDENT AND DEPENDENT VARIABLES******

Independent Variables:
1. Age
2. Sex
3. Region
4. BMI
5. Smoker
6. Children

Dependent Variables:
1. Charges

In [ ]:
X=df_clean[['age','bmi','children','sex','smoker', 'region']]
y=df_clean['charges']


In [ ]:
train_X,test_X,train_y,test_y=train_test_split(X,y,random_state=1)
print(train_X,test_X)

In [ ]:
print(len(train_X),len(train_y))
print(len(test_X),len(test_y))

In [ ]:
def regression(tsX,tsY,trX,trY):
    lrr=LinearRegression()
    rfr=RandomForestRegressor()
    dtr=DecisionTreeRegressor()
    regre=[lrr,rfr,dtr]
    predict=[0,1,2,]
    for index, i in enumerate (regre):
        i.fit(trX,trY)
        predict[index]=i.predict(tsX)
    print('r2_score_lrrr ', r2_score(tsY,predict[0]),'mse:',mse(tsY,predict[0])**0.5,'lrr.coef_:' ,lrr.coef_, 'lrr.intercept:',lrr.intercept_)
    print('r2_score_rfr: ',r2_score(tsY,predict[1]),'mse:',mse(tsY,predict[1])**0.5)
    print('r2_score_dtr: ',r2_score(tsY,predict[2]),'mse:',mse(tsY,predict[2])**0.5)

In [ ]:
regression(test_X,test_y,train_X,train_y)

In [ ]:
max_depth= [10,80, 90, 100, 110]
max_features= [2,3,5]
n_estimators= [100, 200, 300, 1000]

In [ ]:
lrr=LinearRegression().fit(train_X,train_y)

In [ ]:
parameters={
            'max_depth': [10,80, 90, 100, 110],
            'max_features': [2,3,5],
            'n_estimators': [100, 200, 300, 1000] 
}

In [ ]:
rfr=RandomForestRegressor()
grid=GridSearchCV(rfr,parameters,cv=5,n_jobs=-1)
grid.fit(train_X,train_y)
grid.best_params_
grid.best_score_

rfr=RandomForestRegressor()
grid=GridSearchCV(rfr,parameters,cv=5,n_jobs=-1)
grid.fit(train_X,train_y)
grid.best_params_
grid.best_score_

In [ ]:
rfr= RandomForestRegressor(max_depth=10,max_features=3,n_estimators= 1000)
rfr.fit(train_X,train_y)


In [ ]:
predict_Y_rfr=rfr.predict(test_X)
r2_score(test_y,predict_Y_rfr) 

In [ ]:
predict_Y=lrr.predict(test_X)

In [ ]:
r2_score(test_y, predict_Y)

In [ ]:
new_test_data=test_X
new_test_data['Charge']=test_y

In [ ]:
test_y

In [ ]:
len(new_test_data)

In [ ]:
len(predict_Y)

In [ ]:
new_test_data['Predict Charges']=predict_Y

In [ ]:
new_test_data